In [1]:
#importing pandas and numpy and creating a reference variable bgroup to access dataset 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, plot_roc_curve, auc


df = pd.read_csv("C:\\Users\\avula\\Desktop\\normal\\mydataset.csv")
#display the dataset
print(df)

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df.shape

In [ ]:
df.describe(include='all')

In [ ]:
df

In [ ]:
#Displaying indexes 
df.reset_index(level=0, inplace=True)
df

In [ ]:
df.drop_duplicates(subset=None, inplace=True)
df

In [ ]:
# Remove irrelevant columns
dff = df[['Patient_id' , 'Recipient', 'CFF_R', 'HIV_R', 'AIDS_R', 'Risk', 'Age_R']]


In [ ]:
# Split the data into training and test sets
X = dff.drop('Risk', axis=1)
y = dff['Risk']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Encode categorical variables
encoder = OneHotEncoder()
X_train_enc = encoder.fit_transform(X_train[['Recipient']])
X_test_enc = encoder.transform(X_test[['Recipient']])

In [ ]:
# Normalize numerical variables
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[['Age_R']])
X_test_num = scaler.transform(X_test[['Age_R']])

In [ ]:
# Concatenate encoded and normalized features
X_train_final = pd.concat([pd.DataFrame(X_train_enc.toarray()), X_train[['CFF_R', 'HIV_R', 'AIDS_R']].reset_index(drop=True), pd.DataFrame(X_train_num, columns=['Age_R'])], axis=1)
X_test_final = pd.concat([pd.DataFrame(X_test_enc.toarray()), X_test[['CFF_R', 'HIV_R', 'AIDS_R']].reset_index(drop=True), pd.DataFrame(X_test_num, columns=['Age_R'])], axis=1)

In [ ]:
X_train_final

In [ ]:
X_test_final

In [ ]:
# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train_final, y_train)


In [ ]:
# Evaluate the model
accuracy = model.score(X_test_final, y_test)
print('Accuracy:', accuracy)


In [ ]:
# Explore the distribution of the target variable
sns.histplot(data=df, x='Risk', bins=20)
plt.show()

In [ ]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test_final)
conf_mat = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_mat, annot=True, fmt='d')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()

In [ ]:
# Plot the ROC curve
# plot_roc_curve(model, X_test_final, y_test)
plt.show()

In [ ]:
# Plot the feature importances
importance = model.coef_[0]
features = X_train_final.columns
df_importance = pd.DataFrame({'Feature': features, 'Importance': importance})
sns.barplot(data=df_importance, x='Importance', y='Feature')
plt.show()

In [ ]:
# Predict the risk for the given values
given_values = pd.DataFrame({'Recipient': ['A-'], 'CFF_R': [0], 'HIV_R': [0], 'AIDS_R': [0], 'Age_R': [30]})
given_values_enc = encoder.transform(given_values[['Recipient']])
given_values_num = scaler.transform(given_values[['Age_R']])
given_values_final = pd.concat([pd.DataFrame(given_values_enc.toarray()), given_values[['CFF_R', 'HIV_R', 'AIDS_R']].reset_index(drop=True), pd.DataFrame(given_values_num, columns=['Age_R'])], axis=1)
risk = model.predict_proba(given_values_final)[:,1][0]
print('Risk:', risk)

In [ ]:
print("Shape of original dataset:", dff.shape)
print("Shape of test set:", X_test.shape)

In [ ]:
# Filter the original dataset to only include relevant columns and records in X_test
# X_test_final = df.loc[X_test.index, ['Donor', 'Recipient', 'CFF_D', 'HIV_D', 'AIDS_D', 'Risk', 'Age_D', 'Age_R', 'CFF_R', 'HIV_R', 'AIDS_R']]
X_test_final = dff.loc[X_test.index, ['Recipient', 'Risk', 'Age_R', 'CFF_R', 'HIV_R', 'AIDS_R']]


In [ ]:
#Converting blood groups of Donors(Strings) into nums using lambda function 
d = {'A+':1,'A-':2,'B+':3, 'B-':4, 'AB+':5, 'AB-':6, 'O+':7, 'O-':8}
df.Donor = [d[k] for k in df.Donor]
df

In [ ]:
#Converting blood groups of Recipient(Strings) into nums using lambda function 
e = {'A+':1,'A-':2,'B+':3, 'B-':4, 'AB+':5, 'AB-':6, 'O+':7, 'O-':8}
df.Recipient = [e[k] for k in df.Recipient]
df

In [ ]:
# create one-hot encoded version of categorical variables
X_test_final = pd.get_dummies(X_test_final, columns=['Recipient'])


In [ ]:
X_train_final

In [ ]:
del X_test_final["Risk"]
X_test_final

In [ ]:
print(test_set_sorted.columns)


In [ ]:
#Displaying indexes 
# test_set_sorted.reset_index(level=0, inplace=True)
test_set_sorted

In [ ]:
# Plot ROC curve
# plt.plot(fpr, tpr, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
test_set = X_test_final.copy()
test_set['Risk'] = y_test
test_set['Predicted_risk'] = model.predict_proba(X_test_final)[:, 1]
test_set_sorted = test_set.sort_values(by='Predicted_risk', ascending=True)
patient_id_least_risk = test_set_sorted.index[0]
print('Patient ID with least risk:', patient_id_least_risk)